In [21]:
import os.path
import random
import time
from functools import partial
from tkinter import *
import heapq
import sys

from lib.EightPuzzle import EightPuzzle

In [22]:
root = Tk()

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle = EightPuzzle(tuple(state))
solution = None

b = [None] * 9

In [23]:





# def solve():
#     nodes_traversed = 0

#     """
#     Solves the puzzle using Greedy Best-First Search with Manhattan distance heuristic
#     """
#     global puzzle

    

#     #priority queue for open states
#     open_set = [(puzzle.h1(puzzle.initial), puzzle.initial)]
#     print(open_set)
#     # Define a dictionary to store the parent of each state along with the action taken
#     parent = {puzzle.initial: (None, None)}

#     while open_set:
      
#         _, current_state = open_set.pop(0)

#         nodes_traversed += 1

#         print(nodes_traversed)

#         #goal state chceking and returning
#         if puzzle.goal_test(current_state):
#             #reconstructing the path from the goal state to the initial state
#             path = []
#             while parent[current_state][0] is not None:
#                 path.insert(0, parent[current_state][1])
#                 current_state = parent[current_state][0]



    #         print("space complexity is: ")  

    #         print(len(open_set) + nodes_traversed)

    #         print("time complexity is:",nodes_traversed)



            
    #         return path

    #     #getting the neighbors of the current state
    #     for action in puzzle.actions(current_state):
    #         neighbor_state = puzzle.result(current_state, action)
    #         # Add the neighbor state to the open set if it's not already there
    #         if neighbor_state not in parent:
    #             open_set.append((puzzle.h1(neighbor_state), neighbor_state))
    #             parent[neighbor_state] = (current_state, action)

    #     #resorting
    #     open_set.sort(key=lambda x: x[0])
    #     print(open_set)

    # return []

    








def solve():

    nodes_traversed = 0
    """
    Solves the puzzle using A* algorithm with Manhattan distance heuristic
    """
    global puzzle


  



    #making a priority queue
    open_set = [(puzzle.h1(puzzle.initial), 0, puzzle.initial)]
    heapq.heapify(open_set) 
    g_score = {puzzle.initial: 0}
    parent = {puzzle.initial: (None, None)}

    while open_set:
        flag=TRUE
        #heap will return the node with the lowest score
        _, g, current_state = heapq.heappop(open_set)
        nodes_traversed += 1


        print(f"Current State: {current_state}, g-score: {g}")
        print(nodes_traversed)




        #checking if the current state is the goal state and returning
        if puzzle.goal_test(current_state):
            # Reconstruct the path from the goal state to the initial state
            path = []
            while parent[current_state][0] is not None:
                path.insert(0, parent[current_state][1])
                current_state = parent[current_state][0]


            print("space complexity is: ")  

            print(len(open_set) + nodes_traversed)

            print("time complexity is:",nodes_traversed)
           
            return path

        #neighbours of the current node for triversal
        for action in puzzle.actions(current_state):
            neighbor_state = puzzle.result(current_state, action)
            #calculating the tantaive cost for the cost and updating
            tentative_g = g + get_cost(action)

          


            print(f"Neighbor State: {neighbor_state}, tentative g-score: {tentative_g}")






    
            if neighbor_state not in g_score or tentative_g < g_score[neighbor_state]:
                g_score[neighbor_state] = tentative_g
                f_score = tentative_g + puzzle.h1(neighbor_state)
                heapq.heappush(open_set, (f_score, tentative_g, neighbor_state))
                #updating the parent
                parent[neighbor_state] = (current_state, action)




    print(open_set)



    return []  #this will return an empty list if dont find anyting

def get_cost(action):
    """for returning the cost of a given action"""
    if action == 'left':
        return 1
    elif action == 'up':
        return 2
    elif action == 'down':
        return 3
    elif action == 'right':
        return 4
    else:
        return float('inf')  #for handlong the invalid actions
    






In [24]:


def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution = solve()
    print(solution)

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.75))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    # TODO: Find a way to use grid_forget() with a for loop for initialization
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 0))
    b[0].grid(row=0, column=0, ipady=40)
    b[1] = Button(root, text=f'{state[1]}' if state[1] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 1))
    b[1].grid(row=0, column=1, ipady=40)
    b[2] = Button(root, text=f'{state[2]}' if state[2] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 2))
    b[2].grid(row=0, column=2, ipady=40)
    b[3] = Button(root, text=f'{state[3]}' if state[3] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 3))
    b[3].grid(row=1, column=0, ipady=40)
    b[4] = Button(root, text=f'{state[4]}' if state[4] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 4))
    b[4].grid(row=1, column=1, ipady=40)
    b[5] = Button(root, text=f'{state[5]}' if state[5] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 5))
    b[5].grid(row=1, column=2, ipady=40)
    b[6] = Button(root, text=f'{state[6]}' if state[6] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 6))
    b[6].grid(row=2, column=0, ipady=40)
    b[7] = Button(root, text=f'{state[7]}' if state[7] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 7))
    b[7].grid(row=2, column=1, ipady=40)
    b[8] = Button(root, text=f'{state[8]}' if state[8] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 8))
    b[8].grid(row=2, column=2, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(init))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution

    state = [1, 2, 4, 3, 8, 6, 7, 5, 0]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()


Current State: (0, 2, 4, 1, 3, 6, 7, 8, 5), g-score: 0
1
Neighbor State: (1, 2, 4, 0, 3, 6, 7, 8, 5), tentative g-score: inf
Neighbor State: (2, 0, 4, 1, 3, 6, 7, 8, 5), tentative g-score: inf
Current State: (1, 2, 4, 0, 3, 6, 7, 8, 5), g-score: inf
2
Neighbor State: (0, 2, 4, 1, 3, 6, 7, 8, 5), tentative g-score: inf
Neighbor State: (1, 2, 4, 7, 3, 6, 0, 8, 5), tentative g-score: inf
Neighbor State: (1, 2, 4, 3, 0, 6, 7, 8, 5), tentative g-score: inf
Current State: (1, 2, 4, 3, 0, 6, 7, 8, 5), g-score: inf
3
Neighbor State: (1, 0, 4, 3, 2, 6, 7, 8, 5), tentative g-score: inf
Neighbor State: (1, 2, 4, 3, 8, 6, 7, 0, 5), tentative g-score: inf
Neighbor State: (1, 2, 4, 0, 3, 6, 7, 8, 5), tentative g-score: inf
Neighbor State: (1, 2, 4, 3, 6, 0, 7, 8, 5), tentative g-score: inf
Current State: (1, 0, 4, 3, 2, 6, 7, 8, 5), g-score: inf
4
Neighbor State: (1, 2, 4, 3, 0, 6, 7, 8, 5), tentative g-score: inf
Neighbor State: (0, 1, 4, 3, 2, 6, 7, 8, 5), tentative g-score: inf
Neighbor State: (1

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/2f/y2tlbj9j57d3rw3j7zqc2t2c0000gn/T/ipykernel_75394/3109907646.py", line 29, in solve_steps
    create_buttons()
  File "/var/folders/2f/y2tlbj9j57d3rw3j7zqc2t2c0000gn/T/ipykernel_75394/3109907646.py", line 74, in create_buttons
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py", line 2707, in __init__
    Widget.__init__(self, master, 'button', cnf, kw)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py", line 2629, in __in